In [ ]:
import pandas as pd
import yfinance as yf
import os
import sys
from pathlib import Path

project_root = Path(".").resolve().parent

if not (project_root / "src").exists():
    project_root = Path(".").resolve()

if str(project_root) not in sys.path:
    sys.path.append(str(project_root))

pd.set_option("display.max_rows", 100)
pd.set_option("display.max_columns", 50)

print(
    f"Pandas version: {pd.__version__}\
    \nyfinance version: {yf.__version__}\
    \nProject root folder: {project_root}\
    \nCurrent workspace dir: {os.getcwd()}"
)

try:
    from src.config import settings

    print(
        f"\nSettings loaded successfully.\
        \nLoaded {len(settings.INDICES)} indices from config.\
        \nCurrency: {settings.TARGET_CURRENCY}\
        \nTimezone: {settings.TIMEZONE}"
    )
except ImportError as e:
    print(
        f"Cannot load settings: {e}\
        \nEnsure that the notebook starting from root folder of the project."
    )
    settings = None
except Exception as e:
    print(f"Unexpected config load error: {e}")
    settings = None

In [ ]:
if settings and settings.INDICES:
    test_tickers = [settings.INDICES[1]["ticker"], settings.INDICES[3]["ticker"]]
    print(f"Selected tickers: {test_tickers}")
else:
    print("Cannot select tickers, settings not loaded.")
    test_tickers = []

if test_tickers:
    try:
        data = yf.download(
            tickers=test_tickers,
            period="1mo",
            interval="1d",
            group_by="ticker",
            progress=True,
        )
        display(data.head())
        print("\nDataFrame Info:")
        data.info()
    except Exception as e:
        print(f"\nAn error occurred during data download: {e}")
        data = None
else:
    print("No tickers to download data for.")
    data = None

In [ ]:
if data is not None:
    print(f"Index timezone: {data.index.tz}")
    print(f"Using type(): {type(data.index)}")
    if isinstance(data.columns, pd.MultiIndex):
        for ticker in test_tickers:
            print(f"Data for {ticker}")
            ticker_data = data[ticker]
            display(ticker_data.head())
            print(
                f"Is there NaN values? {ticker_data.isnull().values.any()}\
                \nCount of NaN per column:\n{ticker_data.isnull().sum()}"
            )
    else:
        print(
            f"\nIs there NaN values? {data.isnull().values.any()}\
                \nCount of NaN per column:\n{data.isnull().sum()}"
        )

    print("\nTrying intraday data (last 5 days, interval 60m)")
    try:
        intraday_data = yf.download(
            tickers=test_tickers,
            period="5d",
            interval="60m",
            group_by="ticker",
            progress=True,
        )
        display(intraday_data.head())
        print("Intraday DataFrame info:")
        intraday_data.info()
        print(f"\nIntraday Index timezone: {intraday_data.index.tz}")
        if isinstance(intraday_data.columns, pd.MultiIndex):
            for ticker in test_tickers:
                print(f"\nTimezone for {ticker}: {intraday_data[ticker].index.tz}")
                first_ts = intraday_data[ticker].index.min()
                last_ts = intraday_data[ticker].index.max()
                print(f"Data range for {ticker}: {first_ts} to {last_ts}")
    except Exception as e:
        print(f"\nAn error occurred during intraday data download: {e}")
        intraday_data = None

else:
    print("\nCannot analyze data, download failed in the previous step.")

In [ ]:
if settings and settings.INDICES:
    print("Fetching metadata (.info)")
    for index_info_config in settings.INDICES:
        ticker_symbol = index_info_config["ticker"]
        print(f"\nFetching info for {index_info_config['name']} {ticker_symbol}")
        try:
            ticker_obj = yf.Ticker(ticker_symbol)
            info = ticker_obj.info
            print(
                f"  > Name (from yfinance): {info.get('shortName', 'N/A')} / {info.get('longName', 'N/A')}\
                  \n> Currency (from yfinance): {info.get('currency', 'N/A')}\
                  \n> Exchange (from yfinance): {info.get('exchange', 'N/A')}\
                  \n> Timezone (from yfinance): {info.get('exchangeTimezoneName', 'N/A')}\
                  \n> Market State: {info.get('marketState', 'N/A')}\
                  \n> Currency (from config): {index_info_config['currency']}\
                  \n> Exchange (from config): {index_info_config['exchange']}"
            )

            config_exchange_field = index_info_config.get("yfinance_info_field")
            if config_exchange_field:
                print(
                    f"> Exchange (using config field '{config_exchange_field}'): {info.get(config_exchange_field, 'N/A')}"
                )
        except Exception as e:
            print(f"Cannot fetch info for {ticker_symbol}: {e}")
else:
    print("Cannot fetch metadata, settings not loaded.")